# SwellSight Inference

This notebook demonstrates how to use trained SwellSight models for wave analysis inference.

## Purpose
- Load trained models and vocabularies
- Run inference on single images
- Batch inference on multiple images
- Visualize predictions and confidence
- Error analysis and model interpretation

In [ ]:
import os
import json
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont
from pathlib import Path
import pandas as pd
from tqdm.notebook import tqdm
import seaborn as sns
from typing import Dict, List, Tuple, Optional

In [ ]:
# Import components from previous notebooks
%run 01_model_architecture.ipynb
%run 02_data_loading.ipynb
%run 03_loss_and_metrics.ipynb

## Configuration and Model Loading

In [ ]:
# Configuration
INFERENCE_CONFIG = {
    "model_path": "runs/swell_real/best.pt",  # Update this path
    "vocabs_path": "runs/swell_real/vocabs.json",  # Update this path
    "image_size": 224,
    "device": "cuda" if torch.cuda.is_available() else "cpu",
    "batch_size": 32,
}

print("Inference Configuration:")
for key, value in INFERENCE_CONFIG.items():
    print(f"  {key}: {value}")

In [ ]:
def load_model_and_vocabs(model_path: str, vocabs_path: str, device: str):
    """Load trained model and vocabularies."""
    
    # Load vocabularies
    try:
        with open(vocabs_path, "r", encoding="utf-8") as f:
            vocabs = json.load(f)
        wt2id = vocabs["wave_type_to_id"]
        d2id = vocabs["direction_to_id"]
        print(f"✓ Loaded vocabularies from {vocabs_path}")
    except FileNotFoundError:
        print(f"✗ Vocabularies not found: {vocabs_path}")
        print("Using default vocabularies...")
        wt2id = {"beach_break": 0, "reef_break": 1, "point_break": 2, "closeout": 3, "a_frame": 4}
        d2id = {"left": 0, "right": 1, "both": 2}
        vocabs = {"wave_type_to_id": wt2id, "direction_to_id": d2id}
    
    # Create reverse mappings
    id2wt = {v: k for k, v in wt2id.items()}
    id2d = {v: k for k, v in d2id.items()}
    
    # Create model
    model = SwellSightNet(len(wt2id), len(d2id)).to(device)
    
    # Load model weights
    try:
        checkpoint = torch.load(model_path, map_location=device)
        model.load_state_dict(checkpoint["model"])
        model.eval()
        print(f"✓ Loaded model from {model_path}")
        if "epoch" in checkpoint:
            print(f"  Model from epoch {checkpoint['epoch']}")
        if "val_loss" in checkpoint:
            print(f"  Validation loss: {checkpoint['val_loss']:.4f}")
    except FileNotFoundError:
        print(f"✗ Model not found: {model_path}")
        print("Using randomly initialized model for demonstration...")
        model.eval()
    
    return model, vocabs, id2wt, id2d

# Load model and vocabularies
model, vocabs, id2wt, id2d = load_model_and_vocabs(
    INFERENCE_CONFIG["model_path"],
    INFERENCE_CONFIG["vocabs_path"],
    INFERENCE_CONFIG["device"]
)

print(f"\nModel loaded with:")
print(f"  Wave types: {list(id2wt.values())}")
print(f"  Directions: {list(id2d.values())}")
print(f"  Parameters: {sum(p.numel() for p in model.parameters()):,}")

## Inference Functions

In [ ]:
def preprocess_image(image_path: str, image_size: int = 224) -> torch.Tensor:
    """Preprocess image for inference."""
    transform = build_infer_transform(image_size)
    
    if isinstance(image_path, str):
        image = Image.open(image_path).convert("RGB")
    else:
        image = image_path  # Already a PIL Image
    
    return transform(image).unsqueeze(0)  # Add batch dimension


@torch.no_grad()
def predict_single_image(
    model: torch.nn.Module,
    image_path: str,
    id2wt: Dict[int, str],
    id2d: Dict[int, str],
    device: str,
    image_size: int = 224
) -> Dict[str, any]:
    """Run inference on a single image."""
    
    # Preprocess image
    x = preprocess_image(image_path, image_size).to(device)
    
    # Run inference
    pred_h, pred_wt, pred_dir = model(x)
    
    # Convert predictions to interpretable format
    height_m = float(pred_h.item())
    
    # Get class probabilities
    wt_probs = torch.softmax(pred_wt, dim=1).cpu().numpy()[0]
    dir_probs = torch.softmax(pred_dir, dim=1).cpu().numpy()[0]
    
    # Get predicted classes
    wt_pred_id = int(torch.argmax(pred_wt, dim=1).item())
    dir_pred_id = int(torch.argmax(pred_dir, dim=1).item())
    
    wave_type = id2wt[wt_pred_id]
    direction = id2d[dir_pred_id]
    
    # Get confidence scores
    wt_confidence = float(wt_probs[wt_pred_id])
    dir_confidence = float(dir_probs[dir_pred_id])
    
    return {
        "height_meters": height_m,
        "wave_type": wave_type,
        "direction": direction,
        "wave_type_confidence": wt_confidence,
        "direction_confidence": dir_confidence,
        "wave_type_probs": {id2wt[i]: float(prob) for i, prob in enumerate(wt_probs)},
        "direction_probs": {id2d[i]: float(prob) for i, prob in enumerate(dir_probs)},
        "raw_predictions": {
            "height": pred_h.cpu().numpy(),
            "wave_type_logits": pred_wt.cpu().numpy(),
            "direction_logits": pred_dir.cpu().numpy()
        }
    }


def create_dummy_image(size: Tuple[int, int] = (512, 512), wave_params: Dict = None) -> Image.Image:
    """Create a dummy wave image for demonstration."""
    # Create a simple gradient that looks like water
    img = Image.new('RGB', size, color='lightblue')
    draw = ImageDraw.Draw(img)
    
    # Add some wave-like patterns
    for y in range(0, size[1], 20):
        wave_offset = int(10 * np.sin(y * 0.1))
        color_intensity = int(200 + 30 * np.sin(y * 0.05))
        color = (100, 150, color_intensity)
        draw.line([(0, y + wave_offset), (size[0], y + wave_offset)], fill=color, width=3)
    
    # Add text if wave parameters provided
    if wave_params:
        try:
            font = ImageFont.truetype("arial.ttf", 20)
        except:
            font = ImageFont.load_default()
        
        text = f"Dummy Wave\nHeight: {wave_params.get('height', 1.5):.1f}m\nType: {wave_params.get('type', 'beach_break')}"
        draw.text((10, 10), text, fill='white', font=font)
    
    return img

print("✓ Inference functions defined")

## Single Image Inference Demo

In [ ]:
# Create or load a demo image
demo_image_path = "demo_wave.jpg"

# Check if we have a real image, otherwise create a dummy one
if not os.path.exists(demo_image_path):
    print("Creating dummy wave image for demonstration...")
    dummy_params = {"height": 1.8, "type": "reef_break", "direction": "left"}
    demo_image = create_dummy_image(wave_params=dummy_params)
    demo_image.save(demo_image_path)
    print(f"✓ Saved dummy image to {demo_image_path}")
else:
    demo_image = Image.open(demo_image_path)
    print(f"✓ Using existing image: {demo_image_path}")

# Display the demo image
plt.figure(figsize=(10, 6))
plt.imshow(demo_image)
plt.title("Demo Wave Image")
plt.axis('off')
plt.show()

print(f"Image size: {demo_image.size}")

In [ ]:
# Run inference on the demo image
print("Running inference on demo image...")

prediction = predict_single_image(
    model=model,
    image_path=demo_image_path,
    id2wt=id2wt,
    id2d=id2d,
    device=INFERENCE_CONFIG["device"],
    image_size=INFERENCE_CONFIG["image_size"]
)

print("\n" + "="*50)
print("WAVE ANALYSIS RESULTS")
print("="*50)
print(f"Wave Height: {prediction['height_meters']:.2f} meters")
print(f"Wave Type: {prediction['wave_type']} (confidence: {prediction['wave_type_confidence']:.3f})")
print(f"Direction: {prediction['direction']} (confidence: {prediction['direction_confidence']:.3f})")

print(f"\nDetailed Wave Type Probabilities:")
for wt, prob in prediction['wave_type_probs'].items():
    print(f"  {wt}: {prob:.3f}")

print(f"\nDetailed Direction Probabilities:")
for direction, prob in prediction['direction_probs'].items():
    print(f"  {direction}: {prob:.3f}")

## Prediction Visualization

In [ ]:
def visualize_prediction(image_path: str, prediction: Dict, figsize: Tuple[int, int] = (15, 10)):
    """Visualize prediction results with probability distributions."""
    
    fig, axes = plt.subplots(2, 2, figsize=figsize)
    
    # Original image
    if isinstance(image_path, str):
        img = Image.open(image_path)
    else:
        img = image_path
    
    axes[0, 0].imshow(img)
    axes[0, 0].set_title('Input Image')
    axes[0, 0].axis('off')
    
    # Wave type probabilities
    wt_names = list(prediction['wave_type_probs'].keys())
    wt_probs = list(prediction['wave_type_probs'].values())
    
    bars1 = axes[0, 1].bar(wt_names, wt_probs, color='skyblue', alpha=0.7)
    axes[0, 1].set_title('Wave Type Probabilities')
    axes[0, 1].set_ylabel('Probability')
    axes[0, 1].tick_params(axis='x', rotation=45)
    
    # Highlight predicted class
    pred_wt_idx = wt_names.index(prediction['wave_type'])
    bars1[pred_wt_idx].set_color('orange')
    bars1[pred_wt_idx].set_alpha(1.0)
    
    # Add probability values on bars
    for i, (bar, prob) in enumerate(zip(bars1, wt_probs)):
        axes[0, 1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                       f'{prob:.3f}', ha='center', va='bottom', fontsize=9)
    
    # Direction probabilities
    dir_names = list(prediction['direction_probs'].keys())
    dir_probs = list(prediction['direction_probs'].values())
    
    bars2 = axes[1, 0].bar(dir_names, dir_probs, color='lightcoral', alpha=0.7)
    axes[1, 0].set_title('Direction Probabilities')
    axes[1, 0].set_ylabel('Probability')
    
    # Highlight predicted class
    pred_dir_idx = dir_names.index(prediction['direction'])
    bars2[pred_dir_idx].set_color('red')
    bars2[pred_dir_idx].set_alpha(1.0)
    
    # Add probability values on bars
    for i, (bar, prob) in enumerate(zip(bars2, dir_probs)):
        axes[1, 0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                       f'{prob:.3f}', ha='center', va='bottom', fontsize=10)
    
    # Summary text
    axes[1, 1].axis('off')
    summary_text = f"""
PREDICTION SUMMARY
{'='*30}

Wave Height: {prediction['height_meters']:.2f} meters

Wave Type: {prediction['wave_type']}
Confidence: {prediction['wave_type_confidence']:.1%}

Direction: {prediction['direction']}
Confidence: {prediction['direction_confidence']:.1%}

Overall Confidence:
{(prediction['wave_type_confidence'] + prediction['direction_confidence'])/2:.1%}
    """
    
    axes[1, 1].text(0.1, 0.9, summary_text, transform=axes[1, 1].transAxes,
                   fontsize=12, verticalalignment='top', fontfamily='monospace',
                   bbox=dict(boxstyle='round', facecolor='lightgray', alpha=0.8))
    
    plt.tight_layout()
    plt.show()

# Visualize the demo prediction
visualize_prediction(demo_image_path, prediction)

## Batch Inference

In [ ]:
def create_batch_demo_images(n_images: int = 6) -> List[str]:
    """Create a batch of demo images with different characteristics."""
    
    demo_params = [
        {"height": 0.8, "type": "beach_break", "direction": "both"},
        {"height": 1.5, "type": "reef_break", "direction": "left"},
        {"height": 2.2, "type": "point_break", "direction": "right"},
        {"height": 1.0, "type": "closeout", "direction": "both"},
        {"height": 1.8, "type": "a_frame", "direction": "both"},
        {"height": 2.5, "type": "reef_break", "direction": "left"},
    ]
    
    image_paths = []
    
    for i, params in enumerate(demo_params[:n_images]):
        image_path = f"demo_batch_{i+1}.jpg"
        
        if not os.path.exists(image_path):
            demo_image = create_dummy_image(wave_params=params)
            demo_image.save(image_path)
        
        image_paths.append(image_path)
    
    return image_paths, demo_params[:n_images]


def batch_inference(image_paths: List[str], model, id2wt, id2d, device, image_size=224) -> List[Dict]:
    """Run inference on a batch of images."""
    
    predictions = []
    
    for image_path in tqdm(image_paths, desc="Processing images"):
        try:
            pred = predict_single_image(model, image_path, id2wt, id2d, device, image_size)
            pred['image_path'] = image_path
            predictions.append(pred)
        except Exception as e:
            print(f"Error processing {image_path}: {e}")
            continue
    
    return predictions


# Create batch demo images
print("Creating batch of demo images...")
batch_image_paths, batch_true_params = create_batch_demo_images(6)
print(f"✓ Created {len(batch_image_paths)} demo images")

# Run batch inference
print("\nRunning batch inference...")
batch_predictions = batch_inference(
    batch_image_paths, model, id2wt, id2d, INFERENCE_CONFIG["device"]
)
print(f"✓ Processed {len(batch_predictions)} images")

In [ ]:
# Visualize batch results
def visualize_batch_results(predictions: List[Dict], true_params: List[Dict] = None):
    """Visualize batch inference results."""
    
    n_images = len(predictions)
    cols = 3
    rows = (n_images + cols - 1) // cols
    
    fig, axes = plt.subplots(rows, cols, figsize=(15, 5*rows))
    if rows == 1:
        axes = axes.reshape(1, -1)
    
    for i, pred in enumerate(predictions):
        row, col = i // cols, i % cols
        
        # Load and display image
        img = Image.open(pred['image_path'])
        axes[row, col].imshow(img)
        
        # Create title with predictions
        title = f"Predicted:\nH: {pred['height_meters']:.1f}m\n"
        title += f"Type: {pred['wave_type']} ({pred['wave_type_confidence']:.2f})\n"
        title += f"Dir: {pred['direction']} ({pred['direction_confidence']:.2f})"
        
        # Add true values if available
        if true_params and i < len(true_params):
            true = true_params[i]
            title += f"\n\nTrue:\nH: {true['height']:.1f}m\n"
            title += f"Type: {true['type']}\nDir: {true['direction']}"
        
        axes[row, col].set_title(title, fontsize=10)
        axes[row, col].axis('off')
    
    # Hide empty subplots
    for i in range(n_images, rows * cols):
        row, col = i // cols, i % cols
        axes[row, col].axis('off')
    
    plt.tight_layout()
    plt.show()

# Visualize batch results
visualize_batch_results(batch_predictions, batch_true_params)

## Batch Analysis and Statistics

In [ ]:
# Create DataFrame for analysis
batch_df = pd.DataFrame([
    {
        'image': os.path.basename(pred['image_path']),
        'pred_height': pred['height_meters'],
        'pred_wave_type': pred['wave_type'],
        'pred_direction': pred['direction'],
        'wt_confidence': pred['wave_type_confidence'],
        'dir_confidence': pred['direction_confidence'],
        'true_height': batch_true_params[i]['height'] if i < len(batch_true_params) else None,
        'true_wave_type': batch_true_params[i]['type'] if i < len(batch_true_params) else None,
        'true_direction': batch_true_params[i]['direction'] if i < len(batch_true_params) else None,
    }
    for i, pred in enumerate(batch_predictions)
])

print("Batch Inference Results:")
print("=" * 80)
print(batch_df.to_string(index=False))

# Calculate accuracy if we have true labels
if 'true_wave_type' in batch_df.columns and batch_df['true_wave_type'].notna().any():
    wt_accuracy = (batch_df['pred_wave_type'] == batch_df['true_wave_type']).mean()
    dir_accuracy = (batch_df['pred_direction'] == batch_df['true_direction']).mean()
    
    height_mae = np.abs(batch_df['pred_height'] - batch_df['true_height']).mean()
    
    print(f"\nBatch Performance:")
    print(f"Wave Type Accuracy: {wt_accuracy:.2%}")
    print(f"Direction Accuracy: {dir_accuracy:.2%}")
    print(f"Height MAE: {height_mae:.3f}m")

# Confidence statistics
print(f"\nConfidence Statistics:")
print(f"Wave Type Confidence - Mean: {batch_df['wt_confidence'].mean():.3f}, Std: {batch_df['wt_confidence'].std():.3f}")
print(f"Direction Confidence - Mean: {batch_df['dir_confidence'].mean():.3f}, Std: {batch_df['dir_confidence'].std():.3f}")

In [ ]:
# Visualize confidence distributions
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Height predictions vs true (if available)
if 'true_height' in batch_df.columns and batch_df['true_height'].notna().any():
    axes[0].scatter(batch_df['true_height'], batch_df['pred_height'], alpha=0.7, s=100)
    min_h, max_h = batch_df[['true_height', 'pred_height']].min().min(), batch_df[['true_height', 'pred_height']].max().max()
    axes[0].plot([min_h, max_h], [min_h, max_h], 'r--', alpha=0.8)
    axes[0].set_xlabel('True Height (m)')
    axes[0].set_ylabel('Predicted Height (m)')
    axes[0].set_title('Height Predictions')
    axes[0].grid(True, alpha=0.3)
else:
    axes[0].hist(batch_df['pred_height'], bins=10, alpha=0.7, edgecolor='black')
    axes[0].set_xlabel('Predicted Height (m)')
    axes[0].set_ylabel('Frequency')
    axes[0].set_title('Height Prediction Distribution')

# Wave type confidence
axes[1].hist(batch_df['wt_confidence'], bins=10, alpha=0.7, color='skyblue', edgecolor='black')
axes[1].set_xlabel('Wave Type Confidence')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Wave Type Confidence Distribution')
axes[1].axvline(batch_df['wt_confidence'].mean(), color='red', linestyle='--', 
               label=f'Mean: {batch_df["wt_confidence"].mean():.3f}')
axes[1].legend()

# Direction confidence
axes[2].hist(batch_df['dir_confidence'], bins=10, alpha=0.7, color='lightcoral', edgecolor='black')
axes[2].set_xlabel('Direction Confidence')
axes[2].set_ylabel('Frequency')
axes[2].set_title('Direction Confidence Distribution')
axes[2].axvline(batch_df['dir_confidence'].mean(), color='red', linestyle='--',
               label=f'Mean: {batch_df["dir_confidence"].mean():.3f}')
axes[2].legend()

plt.tight_layout()
plt.show()

## Model Interpretation and Analysis

In [ ]:
def analyze_prediction_uncertainty(predictions: List[Dict]):
    """Analyze prediction uncertainty and confidence patterns."""
    
    # Extract confidence scores
    wt_confidences = [p['wave_type_confidence'] for p in predictions]
    dir_confidences = [p['direction_confidence'] for p in predictions]
    
    # Calculate entropy for each prediction (measure of uncertainty)
    wt_entropies = []
    dir_entropies = []
    
    for pred in predictions:
        # Wave type entropy
        wt_probs = np.array(list(pred['wave_type_probs'].values()))
        wt_entropy = -np.sum(wt_probs * np.log(wt_probs + 1e-8))
        wt_entropies.append(wt_entropy)
        
        # Direction entropy
        dir_probs = np.array(list(pred['direction_probs'].values()))
        dir_entropy = -np.sum(dir_probs * np.log(dir_probs + 1e-8))
        dir_entropies.append(dir_entropy)
    
    # Visualization
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    
    # Confidence vs Entropy scatter plots
    axes[0, 0].scatter(wt_confidences, wt_entropies, alpha=0.7, s=100)
    axes[0, 0].set_xlabel('Wave Type Confidence')
    axes[0, 0].set_ylabel('Wave Type Entropy')
    axes[0, 0].set_title('Confidence vs Uncertainty (Wave Type)')
    axes[0, 0].grid(True, alpha=0.3)
    
    axes[0, 1].scatter(dir_confidences, dir_entropies, alpha=0.7, s=100, color='orange')
    axes[0, 1].set_xlabel('Direction Confidence')
    axes[0, 1].set_ylabel('Direction Entropy')
    axes[0, 1].set_title('Confidence vs Uncertainty (Direction)')
    axes[0, 1].grid(True, alpha=0.3)
    
    # Distribution of predictions by class
    wt_counts = pd.Series([p['wave_type'] for p in predictions]).value_counts()
    axes[1, 0].bar(wt_counts.index, wt_counts.values, alpha=0.7)
    axes[1, 0].set_title('Wave Type Prediction Distribution')
    axes[1, 0].set_ylabel('Count')
    axes[1, 0].tick_params(axis='x', rotation=45)
    
    dir_counts = pd.Series([p['direction'] for p in predictions]).value_counts()
    axes[1, 1].bar(dir_counts.index, dir_counts.values, alpha=0.7, color='orange')
    axes[1, 1].set_title('Direction Prediction Distribution')
    axes[1, 1].set_ylabel('Count')
    
    plt.tight_layout()
    plt.show()
    
    # Print statistics
    print("Uncertainty Analysis:")
    print(f"Wave Type - Avg Confidence: {np.mean(wt_confidences):.3f}, Avg Entropy: {np.mean(wt_entropies):.3f}")
    print(f"Direction - Avg Confidence: {np.mean(dir_confidences):.3f}, Avg Entropy: {np.mean(dir_entropies):.3f}")
    
    # Find most/least confident predictions
    most_confident_wt = np.argmax(wt_confidences)
    least_confident_wt = np.argmin(wt_confidences)
    
    print(f"\nMost confident wave type prediction: {predictions[most_confident_wt]['wave_type']} ({wt_confidences[most_confident_wt]:.3f})")
    print(f"Least confident wave type prediction: {predictions[least_confident_wt]['wave_type']} ({wt_confidences[least_confident_wt]:.3f})")

# Analyze prediction uncertainty
analyze_prediction_uncertainty(batch_predictions)

## Export Results

In [ ]:
# Save batch results to file
results_dir = "inference_results"
os.makedirs(results_dir, exist_ok=True)

# Save detailed predictions
detailed_results = {
    "config": INFERENCE_CONFIG,
    "model_info": {
        "wave_types": list(id2wt.values()),
        "directions": list(id2d.values()),
        "parameters": sum(p.numel() for p in model.parameters())
    },
    "predictions": batch_predictions,
    "summary_stats": {
        "n_predictions": len(batch_predictions),
        "avg_wt_confidence": float(np.mean([p['wave_type_confidence'] for p in batch_predictions])),
        "avg_dir_confidence": float(np.mean([p['direction_confidence'] for p in batch_predictions])),
        "height_range": [float(min(p['height_meters'] for p in batch_predictions)),
                        float(max(p['height_meters'] for p in batch_predictions))]
    }
}

with open(os.path.join(results_dir, "batch_predictions.json"), "w") as f:
    json.dump(detailed_results, f, indent=2)

# Save CSV summary
batch_df.to_csv(os.path.join(results_dir, "batch_summary.csv"), index=False)

print(f"✓ Results saved to {results_dir}/")
print(f"  - batch_predictions.json: Detailed predictions with probabilities")
print(f"  - batch_summary.csv: Summary table")

# Print final summary
print(f"\n" + "="*60)
print("INFERENCE SESSION SUMMARY")
print("="*60)
print(f"Model: {INFERENCE_CONFIG['model_path']}")
print(f"Device: {INFERENCE_CONFIG['device']}")
print(f"Images processed: {len(batch_predictions)}")
print(f"Average wave type confidence: {np.mean([p['wave_type_confidence'] for p in batch_predictions]):.3f}")
print(f"Average direction confidence: {np.mean([p['direction_confidence'] for p in batch_predictions]):.3f}")
print(f"Height range: {min(p['height_meters'] for p in batch_predictions):.2f}m - {max(p['height_meters'] for p in batch_predictions):.2f}m")
print("="*60)

## Summary

This notebook demonstrates comprehensive inference capabilities for the SwellSight model:

1. **Model Loading**: Load trained models and vocabularies with error handling
2. **Single Image Inference**: Predict wave parameters for individual images
3. **Batch Processing**: Efficient inference on multiple images
4. **Visualization**: Rich visualizations of predictions and confidence scores
5. **Uncertainty Analysis**: Analyze model confidence and prediction uncertainty
6. **Results Export**: Save predictions in multiple formats for further analysis

The inference pipeline provides:
- Wave height predictions in meters
- Wave type classification with confidence scores
- Direction classification with confidence scores
- Detailed probability distributions for all classes
- Uncertainty quantification through entropy analysis

This notebook can be adapted for production use by:
- Adding real image loading and preprocessing
- Implementing GPU batch processing for efficiency
- Adding model ensemble capabilities
- Integrating with web APIs or mobile applications